In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SIR\\Credit-Card-Default-Prediction-Project-Pwskills\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SIR\\Credit-Card-Default-Prediction-Project-Pwskills'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    validation_dir: Path
    STATUS_FILE: Path
    ALL_REQUIRED_FILES: list

In [13]:
from ccdp.constants import *
from ccdp.utils.common import read_yaml, create_directories

In [17]:
#from entity.data_validation_config import DataValidationConfig
from ccdp.constants import *
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir],[config.validation_dir])

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            validation_dir=Path(config.validation_dir),
            STATUS_FILE=Path(config.STATUS_FILE),
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES
        )

        return data_validation_config


In [20]:
import os
import pandas as pd
from pathlib import Path
from ccdp.logging import logger
#from entity.data_validation_config import DataValidationConfig

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        missing_files = []
        for file_name in self.config.ALL_REQUIRED_FILES:
            file_path = self.config.validation_dir / f"{file_name}.csv"
            if not file_path.exists():
                missing_files.append(file_name)
        
        if missing_files:
            logger.error(f"Missing files: {missing_files}")
            return False
        
        logger.info(f"All required files are present: {self.config.ALL_REQUIRED_FILES}")
        return True
    
    def validate_schema(self, file_path: Path, expected_columns: list) -> bool:
        try:
            df = pd.read_csv(file_path)
            actual_columns = df.columns.tolist()
            if actual_columns == expected_columns:
                logger.info(f"Schema validation passed for {file_path}")
                return True
            else:
                logger.error(f"Schema validation failed for {file_path}. Expected {expected_columns}, got {actual_columns}")
                return False
        except Exception as e:
            logger.error(f"Error during schema validation for {file_path}: {e}")
            return False

    def validate_data(self) -> bool:
        # Check if all files exist
        validation_status = self.validate_all_files_exist()

        if validation_status:
            # Validate schema of each file if all files exist
            expected_columns = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", 
                                "PAY_SEPT", "PAY_AUG", "PAY_JUL", "PAY_JUN", "PAY_MAY", "PAY_APR",
                                "BILL_AMT_SEPT", "BILL_AMT_AUG", "BILL_AMT_JUL", "BILL_AMT_JUN", "BILL_AMT_MAY", "BILL_AMT_APR",
                                "PAY_AMT_SEPT", "PAY_AMT_AUG", "PAY_AMT_JUL", "PAY_AMT_JUN", "PAY_AMT_MAY", "PAY_AMT_APR",
                                "default"]

            for file_name in self.config.ALL_REQUIRED_FILES:
                file_path = self.config.validation_dir / f"{file_name}.csv"
                if not self.validate_schema(file_path, expected_columns):
                    validation_status = False
                    break

        # Ensure the artifacts/data_validation directory exists
        os.makedirs(os.path.dirname(self.config.STATUS_FILE), exist_ok=True)

        # Save validation status to status.txt
        self.save_validation_status(validation_status)

        if validation_status:
            logger.info("Data validation passed for all files.")
        else:
            logger.error("Data validation failed.")
        
        return validation_status
    
    def save_validation_status(self, validation_status: bool):
        status_file_path = self.config.STATUS_FILE
        with open(status_file_path, "w") as status_file:
            status_file.write(f"Validation status: {validation_status}")
        logger.info(f"Validation status saved to {status_file_path}")


In [21]:
#rom component.data_validation import DataValidation
#from config.configuration import ConfigurationManager

try:
    config = ConfigurationManager()

    # Data Ingestion Step
    # data_ingestion_config = config.get_data_ingestion_config()
    # data_ingestion = DataIngestion(config=data_ingestion_config)
    # data_ingestion.download_file()
    # data_ingestion.extract_zip_file()

    # Data Validation Step
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    
    if not data_validation.validate_data():
        raise Exception("Data validation failed!")
    
    logger.info("Data validation succeeded.")

except Exception as e:
    logger.error(f"Pipeline failed due to: {e}")


[2024-08-18 09:47:10,033: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-18 09:47:10,037: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-18 09:47:10,040: INFO: common: created directory at: artifacts]
[2024-08-18 09:47:10,043: INFO: common: created directory at: artifacts/data_validation]
[2024-08-18 09:47:10,047: INFO: 1180685924: All required files are present: ['train', 'test', 'validation']]
[2024-08-18 09:47:10,229: INFO: 1180685924: Schema validation passed for artifacts\data_ingestion\Cleaned_CCD\train.csv]
[2024-08-18 09:47:10,275: INFO: 1180685924: Schema validation passed for artifacts\data_ingestion\Cleaned_CCD\test.csv]
[2024-08-18 09:47:10,310: INFO: 1180685924: Schema validation passed for artifacts\data_ingestion\Cleaned_CCD\validation.csv]
[2024-08-18 09:47:10,313: INFO: 1180685924: Validation status saved to artifacts\data_validation\status.txt]
[2024-08-18 09:47:10,315: INFO: 1180685924: Data validation passed for all fil